In [1]:
%pylab inline
# getting most of my imports and the code for doing the correlations from
# /cmlab/data/fmri/expSamp/anal/ExpSamp_SL_qwarp_log_trim.py
from ExpSamp_SL_qwarp_trans_func import *
from scipy import stats
from joblib import Parallel,delayed
import numpy.lib.recfunctions as nprf 

/data/nielsond/python/envs/py27mvpa/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


/data/nielsond/python/envs/py27mvpa/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [43]:
debug.active += ["SLC"]

# <codecell>
class CustomCorrMeasure(Measure):
    def __init__(self, ind_data, index, use_rank=True, formula = None, method = 'pearson',
                 ind_rank_method = rankdata,
                 s_log = False,t_log = False,e_log = False,
                 s_bounds = None, t_bounds = None,
                 verbose = False,return_term_array = 0,zdist = False,ndmetric = 'correlation'):
        """ for right now, it's up to you to make sure that the index matches your ind_data, 
        for a glm that means you need a separate index for each subject, for an LMER it'll be 
        1 big block of independent data and 1 big index. 
        
        ind_rank_method lets you define a function for ranking the independent data
        this lets us deal with ranking and sorting and interaction terms and all of that"""
        Measure.__init__(self)
        self._ind_data = ind_data[index]
        self._index = index
        self._use_rank = use_rank
        self._formula = formula
        self._method = method
        self._ndmetric = ndmetric
        self._subj = np.unique(self._ind_data['subject'])
        self._is1=True
        self._s_log = s_log
        self._t_log = t_log
        self._e_log = e_log
        self.verbose = verbose
        self._return_term_array = return_term_array
        self._zdist = zdist


        if s_bounds != None:
            self._s_bounds = s_bounds
        if t_bounds != None:
            self._t_bounds = t_bounds
        
        if self._s_log == True:
            self._ind_data['space'] = np.log10(self._ind_data['space'])
            self._ind_data['space'][self._ind_data['space']==np.NINF] =0 

        if self._t_log == True:
            self._ind_data['time'] = np.log10(self._ind_data['time'])
            self._ind_data['time'][self._ind_data['time']==np.NINF] =0 
 
        if self._e_log == True:
            self._ind_data['event'] = np.log10(self._ind_data['event'])
            self._ind_data['event'][self._ind_data['event']==np.NINF] =0
            
        if self._use_rank == True:
            # rank the ind data
            self._ind_data = ind_rank_method(self._ind_data)
        else:
            idat_df = pd.DataFrame(self._ind_data)
            idat_df['val'] = np.zeros(idat_df.shape[0])
            self._ind_data = idat_df.to_records()
        
        #figure out how long the results array will be by calling the glm on some dummy data
        if self._method=='glm':
            self._ind_data['val'] = np.random.randn(self._ind_data.shape[0])     
            self._res_len = (smf.glm(formula=self._formula, data=self._ind_data).fit().params.shape[0]*2)+4
            #set val back to zeros just in case
            self._ind_data['val'] = np.zeros(self._ind_data.shape[0])     
    

    def __call__(self, dataset):
    
        if ((self._method == 'pearson') or (self._method == 'glm')):
            if dataset.shape[1] == 1:
                metric = 'euclidean'
            else:
                metric = self._ndmetric
            # calc dist for neural data
            #mask = dataset.samples[-1].astype(np.bool)
            ndist = pdist(dataset.samples, metric=metric)[self._index]
            if self._zdist == True:
                ndist = (ndist-np.average(ndist))/np.std(ndist)
            if self._use_rank:
                #rank it
                ndist = rankdata(ndist)
            
        if self._method == 'pearson':
            # compare it
            r,p = pearsonr(ndist, self._ind_data[self._formula])
    
            # convert to z
            return np.arctan(r)
        
        elif self._method == 'glm':
            betas = {}
            
            #norm the neural ranks
            #print ndist.shape
            #print type(ndist)
            #print self._subj
            if self._use_rank == True:
                 ndist = ndist.astype(np.float)/np.max(ndist)
            
            #set val in ind_data equal to dep_data for that feature
            self._ind_data['val']=ndist
            #fit glm based on model given in fe_formula
            try:
                modl= smf.glm(formula=self._formula, data=self._ind_data)
                modl.raise_on_perfect_prediction = False
            except (LinAlgError, ValueError):
                return np.zeros(self._res_len)
            try:
                modl = modl.fit()
            except PerfectSeparationError:
                #print "Perfect Separation Error, returning 0 in blind optimism"
                return np.zeros(self._res_len)
                #save beta's for each factor to an array 

            #for fac in modl.pvalues.keys():              
            #    betas[fac]=modl.params[fac]
            betas = np.array([modl.params[fac] for fac in sorted(modl.pvalues.keys())])
            betas = np.append(betas,np.array([modl.tvalues[fac] for fac in sorted(modl.pvalues.keys())]))            
            betas = np.append(betas,modl.deviance)
            betas = np.append(betas,modl.scale)
            betas = np.append(betas,modl.llf)
            betas = np.append(betas, dataset.nfeatures) #this was np.append(betas,mask.sum()) 
            if self.verbose == True:

                if self._is1 == True:
                    k=0            
                    for fac in sorted(modl.pvalues.keys()):
                        print k,fac,"betas"
                        k+=1
                    for fac in sorted(modl.pvalues.keys()):
                        print k,fac,"tvals"
                        k+=1
                    print k,"deviance"
                    print k+1,"scale"
                    print k+2,"llf"
                    print k+3,"count"
                    self._is1=False
            if self._return_term_array == 2:

                terms = []           
                for fac in sorted(modl.pvalues.keys()):
                    terms.append(fac+'_betas')
                for fac in sorted(modl.pvalues.keys()):
                    terms.append(fac+"_tvals")
                terms.append("deviance")
                terms.append("scale")
                terms.append("llf")
                terms.append("count")
                return terms
            elif self._return_term_array == 1:
                return sorted(modl.pvalues.keys())
                
            return betas
        

            
        
        elif self._method == 'lmer':
            raise error('%s method not implemented'%self._method)
        else:
            raise error('%s method not implemented'%self._method)
   

In [3]:
def rank_norm_data(idat,subf='subject',psf='pair_str',tf='time',sf='space',cf='correl',sxtf='spaceXtime'):
    """"rank data man"""
    #rank new independent data in place
    idat_df = pd.DataFrame(idat)
    #print np.unique(idat[psf])
    #print idat_df.columns
    subj = np.unique(idat[subf])
    #print subj
    idat_df['val'] =np.zeros((idat_df.shape[0],1))
    
    
    tl = []
    sl = []
    sxtl = []
    cl = []
    for s in np.unique(idat[subf]):
        tr = rankdata(idat_df.ix[idat_df[subf]==s][tf])
        sr = rankdata(idat_df.ix[idat_df[subf]==s][sf])
        cr = rankdata(idat_df.ix[idat_df[subf]==s][cf])
        sxtr = rankdata(sr*tr)
    
        #norm ranks to be between 0 an 1
        tl = np.append(tl,tr/np.float(np.max(tr)))
        sl = np.append(sl,sr/np.float(np.max(sr)))
        cl = np.append(cl,cr/np.float(np.max(cr)))
        sxtl = np.append(sxtl,sxtr/np.float(np.max(sxtr)))
            
    idat_df[tf]=tl
    idat_df[sf]=sl
    idat_df[cf]=cl
    idat_df[sxtf]=sxtl
            
    return idat_df.to_records()


def addval(idat):
    #rank new independent data in place
    idat_df = pd.DataFrame(idat)
    idat_df['val'] =np.zeros((idat_df.shape[0],1))
    return idat_df.to_records()

In [4]:
# function needed for graphing to change the axes

def adjust_spines(ax, spines,smart_bounds=True):
    for loc, spine in ax.spines.items():
        if loc in spines:
            spine.set_position(('outward', 10))  # outward by 10 points
            spine.set_smart_bounds(smart_bounds)
        else:
            spine.set_color('none')  # don't draw spine

    # turn off ticks where there is no spine
    if 'left' in spines:
        ax.yaxis.set_ticks_position('left')
    else:
        # no yaxis ticks
        ax.yaxis.set_ticks([])

    if 'bottom' in spines:
        ax.xaxis.set_ticks_position('bottom')
    else:
        # no xaxis ticks
        ax.xaxis.set_ticks([])
# Change the default mathtest font to regular instead of italics
# so that the log10s on the graph are regular font
mpl.rcParams['mathtext.default'] = 'regular'

# Change the default font for figures so that they all look the same
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 10}

matplotlib.rc('font', **font)

In [5]:
def log10_nan(x):
    nx = np.log10(x)
    nx[nx==np.NINF] =0
    return nx
def log10_nan_500k(x):
    nx = np.log10(x)
    nx[nx==np.NINF] =0
    return nx + 500000
def log10_nan01(x,xmax,xmin):
    nx = np.log10(x)
    nx[nx==np.NINF] =0
    nxmax = np.log10(xmax)
    nxmin = np.log10(xmin)
    nx = (nx-nxmin)/(nxmax-nxmin)
    return nx
def linear01(x,xmax,xmin):
    nx = (x-xmin)/(xmax-xmin)
    return nx
def exponential01(x,xmax,xmin):
    nx = np.exp((x/xmax))
    nxmax = np.exp(1)
    nxmin = np.exp((xmin/xmax))
    nx = (nx-nxmin)/(nxmax-nxmin)
    return nx

In [3]:
#define formula for glm, this is also a thing that may change run to run
#fe_formula = 'val~ham+ham:viv_ave+viv_ave+correl+event'  # MODEL 1
#run_name = 'perm_hamvivcorrel_stlim_roi_zndist_1000perms_v1' #this one you should probably change for each run

#fe_formula = 'val~ham+correl+event' # MODEL 4
#run_name = 'perm_hamcorrel_stlim_roi_zndist_1000perms_v1'

# this dictionary sets the parameters for the run(s)
lim_dict = dict(hvce=dict(run_name='perm_hamvivcorrel_stlim_roi_zndist_1000perms_v1',
                           fe_formula='val~ham+ham:viv_ave+viv_ave+correl+event',
                           str_only = False,
                           spacemax = 10.**4.5,
                           spacemin = 10**2.0,
                           timemin = 10**4.75,
                           timemax =  10007180.,
                           roi_dir = "standard",
                           ndmetric = "correlation",
                           s_log = False,
                           t_log = False,
                           e_log = True),
                hce=dict(run_name='perm_hamcorrel_stlim_roi_zndist_1000perms_v1',
                           fe_formula='val~ham+correl+event',
                           str_only = False,
                           spacemax = 10.**4.5,
                           spacemin = 10**2.0,
                           timemin = 10**4.75,
                           timemax =  10007180.,
                           roi_dir = "standard",
                           ndmetric = "correlation",
                           s_log = False,
                           t_log = False,
                           e_log = True),
                hvcest=dict(run_name='perm_hamvivcorrelspacetime_stlim_roi_zndist_1000perms_v1',
                           fe_formula='val~ham+ham:viv_ave+space*time+viv_ave+correl+event',
                           str_only = False,
                           spacemax = 10.**4.5,
                           spacemin = 10**2.0,
                           timemin = 10**4.75,
                           timemax =  10007180.,
                           roi_dir = "standard",
                           ndmetric = "correlation",
                           s_log = True,
                           t_log = True,
                           e_log = True),
                hcest=dict(run_name='perm_hamcorrel_stlim_roi_zndist_1000perms_v1',
                           fe_formula='val~ham+correl+space*time+event',
                           str_only = False,
                           spacemax = 10.**4.5,
                           spacemin = 10**2.0,
                           timemin = 10**4.75,
                           timemax =  10007180.,
                           roi_dir = "standard",
                           ndmetric = "correlation",
                           s_log = True,
                           t_log = True,
                           e_log = True)
               )

In [7]:
run_name="temp"

In [8]:
#set all the pathy paths
method = 'glm'
run_type = '%s_sl'%(method)

basedir = '/data/nielsond/expSamp/data/'
data_dir = os.path.join(basedir,'input')
#data_file = 'rsa_dataset_gps_time_old_exclude_viv_bin_scan_time_rem_ham.pickle'  
data_file = 'rsa_dataset_gps_time_old_exclude_drop_viv_bin_scan_time_rem_ham.pickle'  

outdir = '/data/nielsond/expSamp/data/output/%s_sl/%s'%(method,run_name)
ss_out_pat = 'betas.%s.%s.%s.%03d.nii.gz'
combined_out_pat = '%s.%s.%s.nii.gz'
fmri_pat = 'stb.%s_stb_qwarp_e2a.nii.gz'
subj_mask_pat = '%s_mask_GM_full_+tlrc.nii.gz'
errts_pat= 'errts.anaticor.%s_stb_qwarp_e2a+tlrc.nii.gz'
mask_pat='/data/nielsond/expSamp/data/input/MNI152_T1_2.5mm_GM_mask.nii.gz'
stdroidir = os.path.join(data_dir,'all_rois')

subjects = ['expSamp01',
            'expSamp02',
            'expSamp03',
            'expSamp04',
            'expSamp05',
            'expSamp07',
            'expSamp08',
            'expSamp09',
            'expSamp10']

roi_paths = ['LeftAntHippocampus.nii.gz',
             'LeftMidHippocampus.nii.gz',
             'LeftPostHippocampus.nii.gz',
             'Left_Parahippocampal_G.nii.gz',
             'RightAntHippocampus.nii.gz',
             'RightMidHippocampus.nii.gz',
             'RightPostHippocampus.nii.gz',
             'Right_Parahippocampal_G.nii.gz',
             'Left_Posterior_V1.nii.gz',
             'Right_Posterior_V1.nii.gz']
#roi_paths = ['LeftAntHippocampus.nii.gz']

In [9]:
# load independent data
dat = np.load(os.path.join(data_dir,data_file))

# swap strong and weak so that the main effects relflect pair strength strong
# not relevant anymore, don't want to take it out, might break something
# dat['pair_str'] = ['3weak' if dat['pair_str'][i] == '1weak' else '1strong' if dat['pair_str'][i]=='3strong' else dat['pair_str'][i] for i in range(len(dat))]

In [10]:
data_file_old = 'rsa_dataset_gps_time_correl_all.pickle'
dat_old = np.load(os.path.join(data_dir,data_file_old))

In [11]:
cd_new = []
for i in range(len(dat)):
    ind = ((dat_old['subject'] == dat['subject'][i]) & ( dat_old['s1_trial'] == dat['s1_trial'][i]) & (dat_old['s2_trial'] == dat['s2_trial'][i]))
    cd_new.append(dat_old['correl'][ind])

           

In [12]:
dat = nprf.rec_append_fields(dat,'correl',np.array(cd_new).squeeze())

In [13]:
# Drop 118 from expSamp07 (bad single trial beta)
sind = (dat['subject']=='expSamp07') & ((dat['s1_trial']==118) | (dat['s2_trial']==118))
for i in range(len(sind)):
    if sind[i]== True:
        dat[i]['pair_str']='0exclude'
        
dat = dat[dat['pair_str']!='0exclude']

In [14]:
# do some subject level preprocessing so that we can reduce the overhead for each perm
limits = lim_dict.keys()[0]
kept_stims = {}
for subj in subjects:
    ind = ((dat['subject']==subj) & (dat['pair_str'] != '0exclude'))
    s1_exclude = np.array(np.unique(dat[ind]['s1_trial']))
    s2_exclude = np.array(np.unique(dat[ind]['s2_trial']))
    kept_stims[subj] = np.unique(np.hstack([s1_exclude,s2_exclude]))-1


In [18]:
# create roi dictionary
rois = {}
for roi in roi_paths:
    rois[roi]={}
    for subj in subjects:
        rois[roi][subj]= nb.load(os.path.join(stdroidir,'MNI152_2.5mm_'+roi)).get_data()            

In [19]:
# load all the dependent data so that we can iterate over subjects and not have to reload the data each time
# this takes a while depending on how many different runs you are doing, but will save time on each perm
# yes there are probably more memory effecient ways to do this
# please implement one if memory is an issue for you
mask_dict = {subj: nb.load
             (os.path.join(data_dir,subj_mask_pat%(subj))).get_data() for subj in subjects}
roi_mask_dict = {roi:{subj:((mask_dict[subj]>0)&(rois[roi][subj]>0)) for subj in subjects}for roi in roi_paths}
ds_dict = {roi:{subj:fmri_dataset(os.path.join(data_dir,fmri_pat%(subj)),mask=roi_mask_dict[roi][subj]) for subj in subjects}for roi in roi_paths}

 * Please note: warnings are printed only once, but underlying problem might occur many times *


In [15]:
with open('/data/nielsond/expSamp/data/input/ham_perms_all.npz','rb') as handle:
    perm_dict = pickle.load(handle)

In [22]:
# if you want to save youself some time, or anticipate the possiblity of a crash,
# write out the ds dict, I'm writing it out to my home directory so it doesn't clog up teh server
with open(os.path.join(data_dir,'ham_roi_ds_dict.npz'),'wb') as handle:
    pickle.dump(ds_dict,handle)

In [17]:
# if you saved out the ds data in the above cell, and need to recover from a crash,
# Huzzah! you can just load in the dependent data HERE
with open(os.path.join(data_dir,'ham_roi_ds_dict.npz'),'rb') as handle:
    ds_dict = pickle.load(handle)

In [32]:
nperms = 1001

In [46]:
#this will run the perms for all the runs in serial, slow, but doesn't crash
lim_res = {limit:{roi:{subj:[] for subj in subjects}for roi in roi_paths} for limit in lim_dict.keys()}
# Since we're not dropping mixed valence pairs, we can permute before calculating the distance matrix
for limits in lim_dict.keys():
    #for subj in subjects:
    #    #for roi_path in roi_paths:
    print(limits)
    subj = subjects[0]
    roi_path = roi_paths[0]
    ds = ds_dict[roi_path][subj][perm_dict[subj][i]]

    if lim_dict[limits]['str_only'] == True:
        ind = ((dat[dat['subject']==subj]['pair_str']=='1strong') &
                (dat[dat['subject']==subj]['space'] < lim_dict[limits]['spacemax']) & (dat[dat['subject'] == subj]['space'] > lim_dict[limits]['spacemin']) &
                (dat[dat['subject']==subj]['time'] > lim_dict[limits]['timemin']) & (dat[dat['subject']==subj]['time'] < lim_dict[limits]['timemax']))
    else:
        ind = ( 
                (dat[dat['subject']==subj]['space'] < lim_dict[limits]['spacemax']) & (dat[dat['subject'] == subj]['space'] > lim_dict[limits]['spacemin']) &
                (dat[dat['subject']==subj]['time'] > lim_dict[limits]['timemin']) & (dat[dat['subject']==subj]['time'] < lim_dict[limits]['timemax']))

    dsmetric = CustomCorrMeasure(dat[dat['subject']==subj],ind, 
                                 use_rank=False, formula = lim_dict[limits]['fe_formula'], 
                                 method = method,
                                 s_log= True, t_log = True, e_log = True,
                                 zdist = True, ndmetric = lim_dict[limits]['ndmetric'], verbose=True)

    lim_res[limits][roi_path][subj].append(dsmetric.__call__(ds))
print i,


hvcest
0 Intercept betas
1 correl betas
2 event betas
3 ham betas
4 ham:viv_ave betas
5 space betas
6 space:time betas
7 time betas
8 viv_ave betas
9 Intercept tvals
10 correl tvals
11 event tvals
12 ham tvals
13 ham:viv_ave tvals
14 space tvals
15 space:time tvals
16 time tvals
17 viv_ave tvals
18 deviance
19 scale
20 llf
21 count
hvce
0 Intercept betas
1 correl betas
2 event betas
3 ham betas
4 ham:viv_ave betas
5 viv_ave betas
6 Intercept tvals
7 correl tvals
8 event tvals
9 ham tvals
10 ham:viv_ave tvals
11 viv_ave tvals
12 deviance
13 scale
14 llf
15 count
hcest
0 Intercept betas
1 correl betas
2 event betas
3 ham betas
4 space betas
5 space:time betas
6 time betas
7 Intercept tvals
8 correl tvals
9 event tvals
10 ham tvals
11 space tvals
12 space:time tvals
13 time tvals
14 deviance
15 scale
16 llf
17 count
hce
0 Intercept betas
1 correl betas
2 event betas
3 ham betas
4 Intercept tvals
5 correl tvals
6 event tvals
7 ham tvals
8 deviance
9 scale
10 llf
11 count
19


In [24]:
lim_res

{'hce': {'LeftAntHippocampus.nii.gz': {'expSamp01': [array([ -1.51608539e+00,   4.17551857e-01,   3.18998865e-01,
             1.59929591e-03,  -4.29135793e+00,   1.74427246e+00,
             6.78034981e+00,   2.03650048e-01,   2.16669301e+03,
             9.77308529e-01,  -3.12397150e+03,   1.43000000e+02]),
    array([  1.72326093e-01,  -1.44216223e-01,   4.63443812e-02,
            -1.41660460e-02,   4.82187504e-01,  -5.95541259e-01,
             9.73764375e-01,  -1.78319226e+00,   2.21722475e+03,
             1.00010138e+00,  -3.14957325e+03,   1.43000000e+02]),
    array([  7.71966854e-01,  -5.43431793e-01,   2.66794897e-02,
            -1.10485225e-02,   2.16160098e+00,  -2.24571555e+00,
             5.60978579e-01,  -1.39176430e+00,   2.21404165e+03,
             9.98665608e-01,  -3.14797785e+03,   1.43000000e+02])],
   'expSamp02': [array([ -1.49938549e+00,   4.43055933e-01,   2.68293908e-01,
             1.52202484e-02,  -3.87004791e+00,   1.69093971e+00,
             4.375242

In [19]:
#funciton to do the perms in parallel
def run_perm(i,lim_dict,roi_paths,subjects,dat,ds_dict,perm_dict,CustomCorrMeasure):
    lim_res_perm = {limit:{roi:{subj:[] for subj in subjects}for roi in roi_paths} for limit in lim_dict.keys()}

    for limits in lim_dict.keys():
        for subj in subjects:
            for roi_path in roi_paths:
                
                ds = ds_dict[roi_path][subj][perm_dict[subj][i]]
                
                if lim_dict[limits]['str_only'] == True:
                    ind = ((dat[dat['subject']==subj]['pair_str']=='1strong') &
                            (dat[dat['subject']==subj]['space'] < lim_dict[limits]['spacemax']) & (dat[dat['subject'] == subj]['space'] > lim_dict[limits]['spacemin']) &
                            (dat[dat['subject']==subj]['time'] > lim_dict[limits]['timemin']) & (dat[dat['subject']==subj]['time'] < lim_dict[limits]['timemax']))
                else:
                    ind = ( 
                            (dat[dat['subject']==subj]['space'] < lim_dict[limits]['spacemax']) & (dat[dat['subject'] == subj]['space'] > lim_dict[limits]['spacemin']) &
                            (dat[dat['subject']==subj]['time'] > lim_dict[limits]['timemin']) & (dat[dat['subject']==subj]['time'] < lim_dict[limits]['timemax']))
                    
                dsmetric = CustomCorrMeasure(dat[dat['subject']==subj],ind, 
                                             use_rank=False, formula = lim_dict[limits]['fe_formula'], 
                                             method = method,
                                             s_log= True, t_log = True, e_log = True,
                                             zdist = True, ndmetric = lim_dict[limits]['ndmetric'])
                lim_res_perm[limits][roi_path][subj].append(dsmetric.__call__(ds))
    return lim_res_perm

In [ ]:
perm_res = Parallel(n_jobs = 2, verbose = 10)(delayed(run_perm)(i,lim_dict,roi_paths,subjects,dat,ds_dict,perm_dict,CustomCorrMeasure) for i in range(nperms))

[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:   45.3s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:  1.9min
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:  3.6min
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:  5.2min
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:  6.8min
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  8.6min
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 10.6min
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 12.9min
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed: 15.5min
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed: 18.3min
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed: 21.7min
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed: 25.0min
[Parallel(n_jobs=2)]: Done 109 tasks      | elapsed: 28.5min
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed: 32.2min
[Parallel(n_jobs=2)]: Done 141 tasks      | elapsed: 36.5min
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed: 40.6min
[Parallel(n_jobs=2)]: Do

In [37]:
! mkdir /data/nielsond/expSamp/data/output/glm_roi/vishu_res

In [47]:
#convert the data structure from the parallel version to nested dicts

lim_res = {limit:{roi:{subj:[] for subj in subjects}for roi in roi_paths} for limit in lim_dict.keys()}
for i in range(nperms):
    for limits in lim_dict.keys():
        for subj in subjects:
            for roi_path in roi_paths:
                lim_res[limits][roi_path][subj].append(perm_res[i][limits][roi_path][subj])
                
               
for i in range(nperms):
    for limits in lim_dict.keys():
        for subj in subjects:
            for roi_path in roi_paths:
                lim_res[limits][roi_path][subj] = np.array(lim_res[limits][roi_path][subj]).squeeze()

In [25]:
!mkdir /data/nielsond/expSamp/data/output/glm_sl/tse_log_log500k

In [41]:
#save results
with open('/data/nielsond/expSamp/data/output/glm_roi/vishu_res/ham_roi_res_1000.npx','wb') as handle:
    pickle.dump(lim_res,handle)

In [15]:
#load results
with open('/home/dylan/data/fmri/expSamp/data/tse_v1_10k_res_rem.npz','rb') as handle:
    lim_res = pickle.load(handle)

In [52]:
lim_res.keys()
term_dict = dict()
term_dict['hce'] = ["Intercept_betas", "correl_betas", "event_betas", "ham_betas",
                    "Intercept_tvals", "correl_tvals", "event_tvals", "ham_tvals",
                    "deviance", "scale", "llf", "count"]
term_dict['hcest'] = ["Intercept_betas", "correl_betas", "event_betas", "ham_betas", "space_betas", "space:time_betas", "time_betas",
                      "Intercept_tvals", "correl_tvals", "event_tvals", "ham_tvals", "space_tvals", "space:time_tvals", "time_tvals",
                      "deviance", "scale", "llf", "count"]
term_dict['hvce'] = ["Intercept_betas", "correl_betas", "event_betas", "ham_betas", "ham:viv_ave_betas", "viv_ave_betas",
                     "Intercept_tvals", "correl_tvals", "event_tvals", "ham_tvals", "ham:viv_ave_tvals", "viv_ave_tvals",
                     "deviance", "scale", "llf", "count"]
term_dict['hvcest'] = ["Intercept_betas", "correl_betas", "event_betas", "ham_betas", "ham:viv_ave_betas", "space_betas", "space:time_betas", "time_betas", "viv_ave_betas",
                       "Intercept_tvals", "correl_tvals", "event_tvals", "ham_tvals", "ham:viv_ave_tvals", "space_tvals", "space:time_tvals", "time_tvals", "viv_ave_tvals",
                       "deviance", "scale", "llf", "count"]
lim_dict['hce']['terms'] = ["Intercept_betas", "correl_betas", "event_betas", "ham_betas",
                    "Intercept_tvals", "correl_tvals", "event_tvals", "ham_tvals",
                    "deviance", "scale", "llf", "count"]
lim_dict['hcest']['terms'] = ["Intercept_betas", "correl_betas", "event_betas", "ham_betas", "space_betas", "space:time_betas", "time_betas",
                      "Intercept_tvals", "correl_tvals", "event_tvals", "ham_tvals", "space_tvals", "space:time_tvals", "time_tvals",
                      "deviance", "scale", "llf", "count"]
lim_dict['hvce']['terms'] = ["Intercept_betas", "correl_betas", "event_betas", "ham_betas", "ham:viv_ave_betas", "viv_ave_betas",
                     "Intercept_tvals", "correl_tvals", "event_tvals", "ham_tvals", "ham:viv_ave_tvals", "viv_ave_tvals",
                     "deviance", "scale", "llf", "count"]
lim_dict['hvcest']['terms'] = ["Intercept_betas", "correl_betas", "event_betas", "ham_betas", "ham:viv_ave_betas", "space_betas", "space:time_betas", "time_betas", "viv_ave_betas",
                       "Intercept_tvals", "correl_tvals", "event_tvals", "ham_tvals", "ham:viv_ave_tvals", "space_tvals", "space:time_tvals", "time_tvals", "viv_ave_tvals",
                       "deviance", "scale", "llf", "count"]
print ([(k,len(term_dict[k])) for k in term_dict.keys()])

[('hvcest', 22), ('hvce', 16), ('hcest', 18), ('hce', 12)]


In [49]:
# I clearly used to be insane, convert this horible nested dataset into a tidy dataframe
resl = []
for limits in lim_res.keys():
    for subj in subjects:
        for roi_path in lim_res[limits].keys():
            for n in range(len(lim_res[limits][roi_path][subj])):
                for i,t in enumerate(term_dict[limits]):
                    r = {}
                    r['n'] = n
                    r['subject'] = subj
                    r['roi_path'] = roi_path
                    r['model'] = limits
                    r['term'] = t
                    r['value'] = lim_res[limits][roi_path][subj][n][i]
                    resl.append(r)
                
resdf = pd.DataFrame(resl) 

In [51]:
resdf.to_csv('/data/nielsond/expSamp/data/output/glm_roi/vishu_res/ham_roi_res_1000.csv')

In [5]:
resdf = pd.read_csv('/data/nielsond/expSamp/data/output/glm_roi/vishu_res/ham_roi_res_1000.csv', index_col=0)

/data/nielsond/python/envs/py27mvpa/lib/python2.7/site-packages/numpy/lib/arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
resdf.head()

,model,n,roi_path,subject,term,value
0,hvcest,0,Left_Posterior_V1.nii.gz,expSamp01,Intercept_betas,-1.512135
1,hvcest,0,Left_Posterior_V1.nii.gz,expSamp01,correl_betas,0.335867
2,hvcest,0,Left_Posterior_V1.nii.gz,expSamp01,event_betas,0.114491
3,hvcest,0,Left_Posterior_V1.nii.gz,expSamp01,ham_betas,-0.000868
4,hvcest,0,Left_Posterior_V1.nii.gz,expSamp01,ham:viv_ave_betas,0.051770


In [12]:
limits =  lim_dict.keys()[0]

In [7]:
toidf = resdf.loc[resdf.term.str.contains('betas') & ~resdf.term.str.contains('Intercept') & ~resdf.term.str.contains('event') & ~resdf.term.str.contains('correl'),:]

In [8]:
toi_ts = toidf.groupby(['model', 'n','roi_path', 'term']).value.apply(lambda x:stats.ttest_1samp(x, 0)[0])

In [9]:
toi_ts = pd.DataFrame(toi_ts).reset_index()

In [27]:
toi_conj = toi_ts.query('term=="ham_betas"').merge(toi_ts.query('term=="ham:viv_ave_betas"'),
                                        how='right',
                                        on=['model','n','roi_path'],
                                        suffixes=['_ham','_ham:viv_ave'])

In [28]:
toi_conj['value_ham:viv_ave'] *=  -1

In [29]:
toi_conj['value'] = toi_conj.loc[:,['value_ham', 'value_ham:viv_ave']].min(1)

In [41]:
null_dist = sorted(toi_conj.groupby(['n']).value.max().values)
true_res =  toi_conj.query('n == 0')
true_res['all_term_all_roi_p'] = (len(null_dist) - np.searchsorted(null_dist,true_res.value))/(len(null_dist) * 1.0)

/spin1/users/nielsond/python/envs/py27mvpa/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
true_res

,model,n,roi_path,term_ham,value_ham,term_ham:viv_ave,value_ham:viv_ave,value,all_term_all_roi_p
0,hvce,0,LeftAntHippocampus.nii.gz,ham_betas,1.120707,ham:viv_ave_betas,-0.458617,-0.458617,0.998002
1,hvce,0,LeftMidHippocampus.nii.gz,ham_betas,3.659472,ham:viv_ave_betas,0.803610,0.803610,0.695305
2,hvce,0,LeftPostHippocampus.nii.gz,ham_betas,0.876254,ham:viv_ave_betas,0.632704,0.632704,0.776224
3,hvce,0,Left_Parahippocampal_G.nii.gz,ham_betas,1.227161,ham:viv_ave_betas,0.541943,0.541943,0.819181
4,hvce,0,Left_Posterior_V1.nii.gz,ham_betas,0.305826,ham:viv_ave_betas,0.168196,0.168196,0.938062
5,hvce,0,RightAntHippocampus.nii.gz,ham_betas,1.462987,ham:viv_ave_betas,-0.193947,-0.193947,0.984016
6,hvce,0,RightMidHippocampus.nii.gz,ham_betas,2.166649,ham:viv_ave_betas,1.318648,1.318648,0.413586
7,hvce,0,RightPostHippocampus.nii.gz,ham_betas,1.977120,ham:viv_ave_betas,1.648644,1.648644,0.246753
8,hvce,0,Right_Parahippocampal_G.nii.gz,ham_betas,1.976442,ham:viv_ave_betas,1.252307,1.252307,0.453546
9,hvce,0,Right_Posterior_V1.nii.gz,ham_betas,2.646504,ham:viv_ave_betas,2.860631,2.646504,0.044955


In [12]:
toi_ts.query('term=="ham:viv_ave_betas"')

,model,n,roi_path,term,value
50050,hvce,0,LeftAntHippocampus.nii.gz,ham:viv_ave_betas,0.458617
50053,hvce,0,LeftMidHippocampus.nii.gz,ham:viv_ave_betas,-0.803610
50056,hvce,0,LeftPostHippocampus.nii.gz,ham:viv_ave_betas,-0.632704
50059,hvce,0,Left_Parahippocampal_G.nii.gz,ham:viv_ave_betas,-0.541943
50062,hvce,0,Left_Posterior_V1.nii.gz,ham:viv_ave_betas,-0.168196
50065,hvce,0,RightAntHippocampus.nii.gz,ham:viv_ave_betas,0.193947
50068,hvce,0,RightMidHippocampus.nii.gz,ham:viv_ave_betas,-1.318648
50071,hvce,0,RightPostHippocampus.nii.gz,ham:viv_ave_betas,-1.648644
50074,hvce,0,Right_Parahippocampal_G.nii.gz,ham:viv_ave_betas,-1.252307
50077,hvce,0,Right_Posterior_V1.nii.gz,ham:viv_ave_betas,-2.860631


In [ ]:
toi_ts.query('term=="ham_betas"')

In [118]:
toi_ts['value_abs'] = np.abs(toi_ts.value)

In [119]:
null_dist = sorted(toi_ts.groupby('n').value_abs.max().values)

In [120]:
true_res =  toi_ts.query('n == 0')

In [121]:
true_res['all_term_all_roi_p'] = (len(null_dist) - np.searchsorted(null_dist,true_res.value_abs))/(len(null_dist) * 1.0)

/spin1/users/nielsond/python/envs/py27mvpa/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [142]:
true_res.loc[:,['model','roi_path','term','all_term_all_roi_p']]
true_res['ROI'] = true_res.roi_path.str[:-7]

/spin1/users/nielsond/python/envs/py27mvpa/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [143]:
true_res

,model,n,roi_path,term,value,value_abs,all_term_all_roi_p,ROI
0,hce,0,LeftAntHippocampus.nii.gz,ham_betas,1.480078,1.480078,1.000000,LeftAntHippocampus
1,hce,0,LeftMidHippocampus.nii.gz,ham_betas,2.903703,2.903703,0.597403,LeftMidHippocampus
2,hce,0,LeftPostHippocampus.nii.gz,ham_betas,0.418204,0.418204,1.000000,LeftPostHippocampus
3,hce,0,Left_Parahippocampal_G.nii.gz,ham_betas,2.228131,2.228131,0.919081,Left_Parahippocampal_G
4,hce,0,Left_Posterior_V1.nii.gz,ham_betas,0.844220,0.844220,1.000000,Left_Posterior_V1
5,hce,0,RightAntHippocampus.nii.gz,ham_betas,1.608104,1.608104,1.000000,RightAntHippocampus
6,hce,0,RightMidHippocampus.nii.gz,ham_betas,5.082741,5.082741,0.054945,RightMidHippocampus
7,hce,0,RightPostHippocampus.nii.gz,ham_betas,2.589052,2.589052,0.761239,RightPostHippocampus
8,hce,0,Right_Parahippocampal_G.nii.gz,ham_betas,2.382070,2.382070,0.853147,Right_Parahippocampal_G
9,hce,0,Right_Posterior_V1.nii.gz,ham_betas,1.685936,1.685936,0.998002,Right_Posterior_V1


In [53]:
# betas is a nested dict {runs{rois{subjs{[number of perms x number of terms]}}}}
betas = {limits:{roi:{subj:[] for subj in subjects}for roi in lim_res[limits].keys()} for limits in lim_res.keys()}
for limits in lim_res.keys():
    #lim_res[limits] = {k:[] for k in roi_paths}
    for subj in subjects:
        #mask = nb.load(os.path.join(basedir,mask_pat%(subj,subj))).get_data()
        for roi_path in lim_res[limits].keys():
            betas[limits][roi_path][subj]=np.array(lim_res[limits][roi_path][subj])[:,:len(lim_dict[limits]['terms'])]

#real betas is a nested dict {runs{rois{[subjects x perms x terms]}}}
realbetas = {limit:{roi:[] for roi in lim_res[limit].keys()} for limit in lim_dict.keys()}
for limits in lim_dict.keys():
    #lim_res[limits] = {k:[] for k in roi_paths}
    for roi_path in lim_res[limits].keys():
        for subj in subjects:
            realbetas[limits][roi_path].append(betas[limits][roi_path][subj])

In [54]:
# calculate ps and come up with average betas across subjects

avbetas = {limit:{roi:[] for roi in lim_res[limit].keys()} for limit in lim_dict.keys()}

act_avbetas = {limit:{roi:[] for roi in lim_res[limit].keys()} for limit in lim_dict.keys()}
pcent_thresh = {limit:{roi:[] for roi in lim_res[limit].keys()} for limit in lim_dict.keys()}
permps = {limit:{roi:[] for roi in lim_res[limit].keys()} for limit in lim_dict.keys()}
permzs = {limit:{roi:[] for roi in lim_res[limit].keys()} for limit in lim_dict.keys()}

act_permps = {limit:{roi:[] for roi in lim_res[limit].keys()} for limit in lim_dict.keys()}

res_df = pd.DataFrame()
dfi=0
thresh =3
for limits in lim_dict.keys():
    #lim_res[limits] = {k:[] for k in roi_paths}
    for roi_path in lim_res[limits].keys():
        for subj in subjects:
            avbetas[limits][roi_path].append(betas[limits][roi_path][subj])
        act_avbetas[limits][roi_path]= np.average(avbetas[limits][roi_path],axis=0)
        avbetas[limits][roi_path]= stats.ttest_1samp(avbetas[limits][roi_path],0,axis=0)[0]
        #pcent_thresh[limits][roi_path] = np.percentile(np.abs(avbetas[limits][roi_path]),thresh-2,axis=0)
        act_permps[limits][roi_path] = (np.abs(act_avbetas[limits][roi_path][0])<= np.abs(act_avbetas[limits][roi_path])).sum(axis=0)/np.float(act_avbetas[limits][roi_path].shape[0])
        permps[limits][roi_path] = (np.abs(avbetas[limits][roi_path][0])<= np.abs(avbetas[limits][roi_path])).sum(axis=0)/np.float(avbetas[limits][roi_path].shape[0])
        permzs[limits][roi_path] = stats.norm.ppf(permps[limits][roi_path])

        beat_thresh = permps[limits][roi_path] < thresh

        if beat_thresh.sum()>0:
            for j in range(len(beat_thresh)):
                if beat_thresh[j] == True:
                    res_df.loc[dfi,'model'] = limits
                    res_df.loc[dfi,'ROI'] = roi_path[:-7]
                    res_df.loc[dfi,'Term'] = lim_dict[limits]['terms'][j]
                    res_df.loc[dfi,'p'] = permps[limits][roi_path][j]
                    res_df.loc[dfi,'t'] = avbetas[limits][roi_path][0][j]
                    res_df.loc[dfi,'actp'] = act_permps[limits][roi_path][j]
                    res_df.loc[dfi,'beta'] = act_avbetas[limits][roi_path][0][j]
                    print limits+'\t'+roi_path[:-7]+'\t'+lim_dict[limits]['terms'][j]+'\t%0.3g'%permps[limits][roi_path][j]+'\t%0.3g'%avbetas[limits][roi_path][0][j]+'\t%0.3g'%act_avbetas[limits][roi_path][0][j]#,pcent_thresh[limits][roi_path][beat_thresh]
                    dfi+=1

hvcest	Left_Posterior_V1	Intercept_betas	0.222	-1.28	-1.42
hvcest	Left_Posterior_V1	correl_betas	0.807	-0.242	-0.0218
hvcest	Left_Posterior_V1	event_betas	0.000999	9.87	0.23
hvcest	Left_Posterior_V1	ham_betas	0.698	0.397	0.00372
hvcest	Left_Posterior_V1	ham:viv_ave_betas	0.798	-0.241	-0.00296
hvcest	Left_Posterior_V1	space_betas	0.376	0.912	0.296
hvcest	Left_Posterior_V1	space:time_betas	0.272	-1.15	-0.066
hvcest	Left_Posterior_V1	time_betas	0.335	1.01	0.192
hvcest	Left_Posterior_V1	viv_ave_betas	0.386	-0.94	-0.0862
hvcest	Left_Posterior_V1	Intercept_tvals	0.255	-1.18	-0.586
hvcest	Left_Posterior_V1	correl_tvals	0.629	-0.544	-0.226
hvcest	Left_Posterior_V1	event_tvals	0.000999	7.23	5.37
hvcest	Left_Posterior_V1	ham_tvals	0.575	0.614	0.347
hvcest	Left_Posterior_V1	ham:viv_ave_tvals	0.811	-0.238	-0.121
hvcest	Left_Posterior_V1	space_tvals	0.514	0.672	0.366
hvcest	Left_Posterior_V1	space:time_tvals	0.393	-0.891	-0.492
hvcest	Left_Posterior_V1	time_tvals	0.403	0.879	0.445
hvcest	Left_Poste

hvcest	RightAntHippocampus	correl_betas	0.751	0.325	0.0359
hvcest	RightAntHippocampus	event_betas	0.000999	7.22	0.319
hvcest	RightAntHippocampus	ham_betas	0.188	1.47	0.00858
hvcest	RightAntHippocampus	ham:viv_ave_betas	0.824	0.242	0.00342
hvcest	RightAntHippocampus	space_betas	0.0909	1.84	0.555
hvcest	RightAntHippocampus	space:time_betas	0.0859	-1.87	-0.0915
hvcest	RightAntHippocampus	time_betas	0.0579	2.09	0.345
hvcest	RightAntHippocampus	viv_ave_betas	0.548	-0.666	-0.0935
hvcest	RightAntHippocampus	Intercept_tvals	0.00999	-3.23	-1.31
hvcest	RightAntHippocampus	correl_tvals	0.787	0.292	0.164
hvcest	RightAntHippocampus	event_tvals	0.000999	5.5	7.72
hvcest	RightAntHippocampus	ham_tvals	0.219	1.4	0.6
hvcest	RightAntHippocampus	ham:viv_ave_tvals	0.857	0.194	0.117
hvcest	RightAntHippocampus	space_tvals	0.139	1.57	0.673
hvcest	RightAntHippocampus	space:time_tvals	0.142	-1.51	-0.637
hvcest	RightAntHippocampus	time_tvals	0.0909	1.86	0.787
hvcest	RightAntHippocampus	viv_ave_tvals	0.525	-0.705	

hvce	LeftAntHippocampus	Intercept_betas	0.000999	-10	-1.1
hvce	LeftAntHippocampus	correl_betas	0.267	1.19	0.0825
hvce	LeftAntHippocampus	event_betas	0.000999	15.6	0.315
hvce	LeftAntHippocampus	ham_betas	0.305	1.12	0.00365
hvce	LeftAntHippocampus	ham:viv_ave_betas	0.694	0.459	0.00331
hvce	LeftAntHippocampus	viv_ave_betas	0.179	1.6	0.08
hvce	LeftAntHippocampus	Intercept_tvals	0.000999	-9.73	-3.41
hvce	LeftAntHippocampus	correl_tvals	0.231	1.27	0.352
hvce	LeftAntHippocampus	event_tvals	0.000999	12.4	7.27
hvce	LeftAntHippocampus	ham_tvals	0.334	1.03	0.237
hvce	LeftAntHippocampus	ham:viv_ave_tvals	0.716	0.418	0.14
hvce	LeftAntHippocampus	viv_ave_tvals	0.208	1.45	0.446
hvce	LeftAntHippocampus	deviance	0.935	8.71	2.6e+03
hvce	LeftAntHippocampus	scale	0.981	292	0.973
hvce	LeftAntHippocampus	llf	0.934	-8.76	-3.76e+03
hvce	LeftAntHippocampus	count	1	35.9	127
hvce	RightAntHippocampus	Intercept_betas	0.000999	-6.11	-1
hvce	RightAntHippocampus	correl_betas	0.755	0.344	0.0372
hvce	RightAntHippocampu

hcest	Right_Posterior_V1	space_betas	0.25	1.24	0.421
hcest	Right_Posterior_V1	space:time_betas	0.248	-1.26	-0.0727
hcest	Right_Posterior_V1	time_betas	0.327	1.11	0.237
hcest	Right_Posterior_V1	Intercept_tvals	0.215	-1.34	-0.773
hcest	Right_Posterior_V1	correl_tvals	0.485	-0.83	-0.832
hcest	Right_Posterior_V1	event_tvals	0.000999	7.67	4.32
hcest	Right_Posterior_V1	ham_tvals	0.114	1.7	1.59
hcest	Right_Posterior_V1	space_tvals	0.265	1.19	0.606
hcest	Right_Posterior_V1	space:time_tvals	0.257	-1.23	-0.627
hcest	Right_Posterior_V1	time_tvals	0.334	1.05	0.585
hcest	Right_Posterior_V1	deviance	0.409	8.72	2.61e+03
hcest	Right_Posterior_V1	scale	0.859	227	0.975
hcest	Right_Posterior_V1	llf	0.406	-8.76	-3.76e+03
hcest	Right_Posterior_V1	count	1	21.9	95
hcest	LeftAntHippocampus	Intercept_betas	0.002	-5.37	-2.32
hcest	LeftAntHippocampus	correl_betas	0.185	1.45	0.0987
hcest	LeftAntHippocampus	event_betas	0.000999	15.2	0.315
hcest	LeftAntHippocampus	ham_betas	0.227	1.39	0.00524
hcest	LeftAntHippocamp

hce	Right_Posterior_V1	scale	0.991	268	0.986
hce	Right_Posterior_V1	llf	0.018	-8.81	-3.78e+03
hce	Right_Posterior_V1	count	1	21.9	95
hce	LeftAntHippocampus	Intercept_betas	0.000999	-9.99	-1.07
hce	LeftAntHippocampus	correl_betas	0.225	1.34	0.0911
hce	LeftAntHippocampus	event_betas	0.000999	15.5	0.313
hce	LeftAntHippocampus	ham_betas	0.194	1.48	0.00589
hce	LeftAntHippocampus	Intercept_tvals	0.000999	-8.61	-3.51
hce	LeftAntHippocampus	correl_tvals	0.171	1.47	0.395
hce	LeftAntHippocampus	event_tvals	0.000999	12.1	7.22
hce	LeftAntHippocampus	ham_tvals	0.203	1.45	0.808
hce	LeftAntHippocampus	deviance	0.998	8.73	2.61e+03
hce	LeftAntHippocampus	scale	1	301	0.976
hce	LeftAntHippocampus	llf	0.998	-8.76	-3.77e+03
hce	LeftAntHippocampus	count	1	35.9	127
hce	RightAntHippocampus	Intercept_betas	0.000999	-7.45	-1.04
hce	RightAntHippocampus	correl_betas	0.827	0.248	0.028
hce	RightAntHippocampus	event_betas	0.000999	7.23	0.322
hce	RightAntHippocampus	ham_betas	0.16	1.61	0.0081
hce	RightAntHippocampus	

In [62]:
permps

{'hce': {'LeftAntHippocampus.nii.gz': array([  9.99000999e-04,   2.24775225e-01,   9.99000999e-04,
           1.93806194e-01,   9.99000999e-04,   1.70829171e-01,
           9.99000999e-04,   2.02797203e-01,   9.98001998e-01,
           1.00000000e+00,   9.98001998e-01,   1.00000000e+00]),
  'LeftMidHippocampus.nii.gz': array([  9.99000999e-04,   5.18481518e-01,   9.99000999e-04,
           1.89810190e-02,   9.99000999e-04,   5.48451548e-01,
           9.99000999e-04,   1.29870130e-02,   1.00000000e+00,
           1.00000000e+00,   1.00000000e+00,   1.00000000e+00]),
  'LeftPostHippocampus.nii.gz': array([  9.99000999e-04,   6.76323676e-01,   9.99000999e-04,
           6.79320679e-01,   9.99000999e-04,   7.07292707e-01,
           9.99000999e-04,   4.98501499e-01,   9.96003996e-01,
           9.99000999e-01,   9.88011988e-01,   1.00000000e+00]),
  'Left_Parahippocampal_G.nii.gz': array([  9.99000999e-04,   5.58441558e-01,   9.99000999e-04,
           4.89510490e-02,   9.99000999e-04,   

In [129]:
uncor_res = res_df.loc[res_df.Term.str.contains('betas') & ~res_df.Term.str.contains('Intercept') & ~res_df.Term.str.contains('event')& ~res_df.Term.str.contains('correl'),:]

In [130]:
uncor_res['bonferoni_p'] = uncor_res.p * 140

/spin1/users/nielsond/python/envs/py27mvpa/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [145]:
all_res = uncor_res.merge(true_res.loc[:,['model','ROI','term','all_term_all_roi_p']],
                left_on=['model','ROI','Term'],
                right_on = ['model','ROI','term'])

In [150]:
all_res.loc[all_res.bonferoni_p > 1, "bonferoni_p"] = 1

In [ ]:
all_res.loc

In [153]:
all_res.query('p < 0.05')

,model,ROI,Term,p,t,actp,beta,bonferoni_p,term,all_term_all_roi_p,bonbonferoni_p
29,hvcest,RightPostHippocampus,viv_ave_betas,0.018981,3.000215,0.151848,0.116402,1.00000,viv_ave_betas,0.550450,1.0
30,hvcest,Right_Posterior_V1,ham_betas,0.033966,2.641590,0.013986,0.022916,1.00000,ham_betas,0.740260,1.0
31,hvcest,Right_Posterior_V1,ham:viv_ave_betas,0.018981,-3.029674,0.125874,-0.018423,1.00000,ham:viv_ave_betas,0.528472,1.0
38,hvcest,LeftAntHippocampus,space_betas,0.011988,3.170764,0.139860,0.392455,1.00000,space_betas,0.460539,1.0
39,hvcest,LeftAntHippocampus,space:time_betas,0.028971,-2.635573,0.205794,-0.059070,1.00000,space:time_betas,0.743257,1.0
40,hvcest,LeftAntHippocampus,time_betas,0.023976,2.621255,0.259740,0.172693,1.00000,time_betas,0.748252,1.0
54,hvcest,LeftMidHippocampus,ham_betas,0.001998,3.810661,0.002997,0.020510,0.27972,ham_betas,0.234765,NaN
74,hvce,RightPostHippocampus,viv_ave_betas,0.019980,2.911806,0.161838,0.113128,1.00000,viv_ave_betas,0.594406,1.0
75,hvce,Right_Posterior_V1,ham_betas,0.028971,2.646504,0.017982,0.022454,1.00000,ham_betas,0.737263,1.0
76,hvce,Right_Posterior_V1,ham:viv_ave_betas,0.024975,-2.860631,0.139860,-0.017685,1.00000,ham:viv_ave_betas,0.619381,1.0
